# **Data Understanding**

In [1]:
import pandas as pd

# Load the dataset
data = pd.read_csv('./dataset/data.csv', engine='python', encoding='ISO-8859-1')

# Show top 5 datas
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


The Dataset was imported from kaggle. The dataset itself is telling us about simple e-commerce transaction historical data. Here's some details about each column from the dataset,

- **InvoiceNo**: A unique identifier for each transaction or invoice. It helps in tracking sales records and analyzing purchase patterns.
- **StockCode**: A unique code assigned to each product, which can be used to identify and categorize different items in the inventory.
- **Description**: A textual description of the product purchased. This column provides insights into product demand and popular items.
- **Quantity**: The number of units of a product purchased in a specific transaction. Negative values in this column may indicate product returns.
- **InvoiceDate**: The exact date and time when the transaction took place. It is useful for time-series analysis and identifying seasonal trends in sales.
- **UnitPrice**: The price per unit of the product purchased. This column helps in revenue calculations and price trend analysis.
- **CustomerID**: A unique identifier assigned to each customer. It is crucial for customer segmentation, behavior analysis, and retention strategies.
- **Country**: The country where the customer is located. This column is useful for geographic analysis of sales and market segmentation.

In [2]:
# Display basic information about the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


# **Data Cleaning**

In [3]:
# Parsing CustomerID into string, regarding its purpose as identifier
data['CustomerID'] = data['CustomerID'].astype(str)

In [4]:
# Check for duplicated values
total_data = len(data)
total_duplicates = data.duplicated().sum()
percentage_duplicates = (total_duplicates / total_data) * 100
print(percentage_duplicates)

0.9721189350979592


Since we only have few duplicated rows in the dataset and duplicated values itself does not relevant with the business logic, so we can securely drop them.

In [5]:
# Drop duplicated values
data = data.drop_duplicates()
total_data = len(data)
total_duplicates = data.duplicated().sum()
percentage_duplicates = (total_duplicates / total_data) * 100
print(percentage_duplicates)

0.0


In [6]:
# Check for missing values
data.isnull().sum()

InvoiceNo         0
StockCode         0
Description    1454
Quantity          0
InvoiceDate       0
UnitPrice         0
CustomerID        0
Country           0
dtype: int64

In [7]:
# Find invoices where some rows have missing CustomerID and others do not
invoice_with_mixed_ids = data.groupby('InvoiceNo')['CustomerID'].transform(lambda x: x.isnull().any() and x.notnull().any())

# Filter dataset to show only these invoices
data_filtered = data[invoice_with_mixed_ids]

# Display the affected invoices
print("Invoices where some rows have missing CustomerID while others have valid values:")
print(data_filtered[['InvoiceNo', 'CustomerID']].sort_values(by="InvoiceNo").head(20))  # Display first 20 rows for preview

Invoices where some rows have missing CustomerID while others have valid values:
Empty DataFrame
Columns: [InvoiceNo, CustomerID]
Index: []


In [8]:
# Show the number of missing CustomerID values per country
print(data[data['CustomerID'].isnull()]['Country'].value_counts())


Series([], Name: count, dtype: int64)


In [9]:
# Show the Customer Ids with missing value by StockCode
print(data[data['CustomerID'].isnull()]['StockCode'].value_counts().head(10))

Series([], Name: count, dtype: int64)


Since we haven't find the pattern for missing Customer Ids and we don't have enough information about where are the missing values come from, we can assume if those customers who don't have Id as a guest who make only one transaction. 

In [11]:
# Correctly fill missing CustomerID with 'Guest'
data['CustomerID'] = data['CustomerID'].fillna('Guest')

# Correctly fill missing product descriptions with 'Unknown'
data['Description'] = data['Description'].fillna('Unknown')

# Check remaining missing values
print(data.isna().sum())


InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64


In [12]:
# Calculate the percentage of total rows that have minus value for each numeric column
numeric_columns = data.select_dtypes(include=['number']).columns
percentage_negative_values = (data[numeric_columns] < 0).mean() * 100

print(percentage_negative_values)

Quantity     1.972827
UnitPrice    0.000373
dtype: float64


The business logic itself is allowing if the 'Quantity' or 'UnitPrice' to have minus value. It could means the customer is doing the return process. So in that case, we don't have to clean the data.   

# **EDA (Exploratory Data Analysis)**